# Topological Data Analysis - Exploration

Explore building attention graphs and computing persistent homology on encoder attention patterns.

**Methodology:**
1. Extract last layer attention
2. Average across attention heads
3. Filter special tokens and renormalize
4. Symmetrize attention matrix
5. Convert to distance: `d_ij = 1 - attention_ij`
6. Compute Vietoris-Rips persistent homology
7. Extract persistence diagrams (β₀, β₁)

## 1. Install and Import Dependencies

In [ ]:
# Install giotto-tda if not already installed
# !pip install giotto-tda

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# TDA libraries
from gtda.homology import VietorisRipsPersistence
from gtda.plotting import plot_diagram

print("✓ Libraries imported")

## 2. Load Attention Data

In [ ]:
# Load the extracted attention maps
data_path = Path("../data/attention_maps/all_encoder_attention.pkl")

print(f"Loading data from {data_path}...")
with open(data_path, 'rb') as f:
    results = pickle.load(f)

print(f"✓ Loaded {len(results)} sentence pairs")
print(f"\nModel architecture: {results[0]['en_attention'].shape[0]} layers, {results[0]['en_attention'].shape[1]} heads")

## 3. Define Graph Construction Functions

In [ ]:
def build_distance_matrix(attention, tokens, layer=-1):
    """
    Build distance matrix from attention weights.
    
    Args:
        attention: Attention tensor (num_layers, num_heads, seq_len, seq_len)
        tokens: List of token strings
        layer: Which layer to use (-1 for last layer)
    
    Returns:
        distance_matrix: (N, N) array where N = number of content tokens
        content_tokens: List of content token strings
    """
    # 1. Extract last layer and average over heads
    attn = attention[layer].mean(axis=0)  # (seq_len, seq_len)
    
    # 2. Filter special tokens
    special_tokens = {'</s>', '<s>', '<pad>', 'eng_Latn', 'fra_Latn'}
    content_mask = np.array([tok not in special_tokens for tok in tokens])
    
    # Filter attention matrix
    attn_filtered = attn[content_mask][:, content_mask]
    content_tokens = [tok for tok, keep in zip(tokens, content_mask) if keep]
    
    # 3. Renormalize
    row_sums = attn_filtered.sum(axis=1, keepdims=True)
    attn_filtered = attn_filtered / row_sums
    
    # 4. Symmetrize (make undirected)
    attn_sym = (attn_filtered + attn_filtered.T) / 2
    
    # 5. Convert to distance: d = 1 - attention
    distance_matrix = 1 - attn_sym
    
    # Ensure diagonal is 0 and symmetric
    np.fill_diagonal(distance_matrix, 0)
    
    return distance_matrix, content_tokens


print("✓ Function defined: build_distance_matrix()")

## 4. Test on Sample Example

In [ ]:
# Select first example
idx = 0
example = results[idx]

print(f"Example {idx}:")
print(f"English: {example['en_text']}")
print(f"French:  {example['fr_text']}")
print()

# Build distance matrices
en_dist, en_tokens = build_distance_matrix(example['en_attention'], example['en_tokens'])
fr_dist, fr_tokens = build_distance_matrix(example['fr_attention'], example['fr_tokens'])

print(f"English distance matrix: {en_dist.shape}")
print(f"  Content tokens: {en_tokens}")
print(f"  Min distance: {en_dist.min():.4f}")
print(f"  Max distance: {en_dist.max():.4f}")
print(f"  Mean distance: {en_dist.mean():.4f}")
print()

print(f"French distance matrix: {fr_dist.shape}")
print(f"  Content tokens: {fr_tokens}")
print(f"  Min distance: {fr_dist.min():.4f}")
print(f"  Max distance: {fr_dist.max():.4f}")
print(f"  Mean distance: {fr_dist.mean():.4f}")

### Visualize Distance Matrices

In [ ]:
# Plot distance matrices
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# English
sns.heatmap(en_dist, xticklabels=en_tokens, yticklabels=en_tokens, 
            cmap='RdYlBu_r', ax=ax1, square=True,
            cbar_kws={'label': 'Distance (1 - attention)'})
ax1.set_title(f'English Distance Matrix\n{example["en_text"][:50]}...')
ax1.set_xlabel('Tokens')
ax1.set_ylabel('Tokens')
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45, ha='right')

# French
sns.heatmap(fr_dist, xticklabels=fr_tokens, yticklabels=fr_tokens, 
            cmap='RdYlBu_r', ax=ax2, square=True,
            cbar_kws={'label': 'Distance (1 - attention)'})
ax2.set_title(f'French Distance Matrix\n{example["fr_text"][:50]}...')
ax2.set_xlabel('Tokens')
ax2.set_ylabel('Tokens')
plt.setp(ax2.xaxis.get_majorticklabels(), rotation=45, ha='right')

plt.tight_layout()
plt.show()

## 5. Compute Persistent Homology

In [ ]:
# Initialize Vietoris-Rips persistence
# homology_dimensions=[0, 1] computes β₀ (connected components) and β₁ (loops)
VR = VietorisRipsPersistence(homology_dimensions=[0, 1], n_jobs=-1)

print("✓ Vietoris-Rips persistence initialized")
print(f"  Computing: β₀ (connected components) and β₁ (1-dimensional holes/loops)")

In [ ]:
# Compute persistence for English
# Input needs to be 3D: (n_samples, n_points, n_points)
en_diagram = VR.fit_transform([en_dist])[0]  # Extract first (and only) diagram

print("English persistence diagram:")
print(f"  Shape: {en_diagram.shape}")
print(f"  Format: (birth, death, dimension)")
print(f"\nFirst 10 features:")
print(en_diagram[:10])

In [ ]:
# Compute persistence for French
fr_diagram = VR.fit_transform([fr_dist])[0]

print("French persistence diagram:")
print(f"  Shape: {fr_diagram.shape}")
print(f"\nFirst 10 features:")
print(fr_diagram[:10])

### Analyze Persistence Features

In [ ]:
# Count features by dimension
def analyze_diagram(diagram, language=""):
    dim0 = diagram[diagram[:, 2] == 0]  # β₀ features
    dim1 = diagram[diagram[:, 2] == 1]  # β₁ features
    
    # Persistence = death - birth
    persistence0 = dim0[:, 1] - dim0[:, 0]
    persistence1 = dim1[:, 1] - dim1[:, 0]
    
    print(f"{language} Persistence Summary:")
    print("="*50)
    print(f"β₀ features (connected components): {len(dim0)}")
    if len(persistence0) > 0:
        print(f"  Max persistence: {persistence0.max():.4f}")
        print(f"  Mean persistence: {persistence0.mean():.4f}")
    
    print(f"\nβ₁ features (loops/holes): {len(dim1)}")
    if len(persistence1) > 0:
        print(f"  Max persistence: {persistence1.max():.4f}")
        print(f"  Mean persistence: {persistence1.mean():.4f}")
    print()

analyze_diagram(en_diagram, "English")
analyze_diagram(fr_diagram, "French")

## 6. Visualize Persistence Diagrams

In [ ]:
# Plot English persistence diagram
fig = plot_diagram(en_diagram)
plt.title(f'English Persistence Diagram\n{example["en_text"][:60]}...')
plt.tight_layout()
plt.show()

In [ ]:
# Plot French persistence diagram
fig = plot_diagram(fr_diagram)
plt.title(f'French Persistence Diagram\n{example["fr_text"][:60]}...')
plt.tight_layout()
plt.show()

## 7. Compare Persistence Diagrams

Use Wasserstein distance to measure similarity between English and French topologies.

In [ ]:
from gtda.diagrams import PairwiseDistance

# Initialize Wasserstein distance metric
wasserstein = PairwiseDistance(metric='wasserstein', n_jobs=-1)

# Compute distance between English and French diagrams
# Need to reshape to (n_samples, n_features, 3)
diagrams_pair = np.array([en_diagram, fr_diagram])

# Compute pairwise distances
distances = wasserstein.fit_transform(diagrams_pair)

print("Wasserstein Distance Matrix:")
print(distances)
print()
print(f"Wasserstein distance between English and French: {distances[0, 1]:.6f}")

## 8. Process Multiple Examples

In [ ]:
# Process first 10 examples
num_examples = 10
wasserstein_distances = []

print(f"Computing persistent homology for {num_examples} sentence pairs...\n")

for idx in range(num_examples):
    example = results[idx]
    
    # Build distance matrices
    en_dist, _ = build_distance_matrix(example['en_attention'], example['en_tokens'])
    fr_dist, _ = build_distance_matrix(example['fr_attention'], example['fr_tokens'])
    
    # Compute persistence
    en_diagram = VR.fit_transform([en_dist])[0]
    fr_diagram = VR.fit_transform([fr_dist])[0]
    
    # Compute Wasserstein distance
    diagrams_pair = np.array([en_diagram, fr_diagram])
    dist_matrix = wasserstein.fit_transform(diagrams_pair)
    w_dist = dist_matrix[0, 1]
    
    wasserstein_distances.append({
        'idx': idx,
        'en_text': example['en_text'],
        'fr_text': example['fr_text'],
        'wasserstein_distance': w_dist,
        'en_diagram': en_diagram,
        'fr_diagram': fr_diagram
    })
    
    print(f"[{idx}] W-dist: {w_dist:.6f} | EN: {example['en_text'][:40]}...")

print(f"\n✓ Processed {num_examples} examples")

In [ ]:
# Summary statistics
w_dists = [d['wasserstein_distance'] for d in wasserstein_distances]

print("Wasserstein Distance Statistics (first 10 pairs):")
print("="*50)
print(f"Min:    {np.min(w_dists):.6f}")
print(f"Max:    {np.max(w_dists):.6f}")
print(f"Mean:   {np.mean(w_dists):.6f}")
print(f"Median: {np.median(w_dists):.6f}")
print(f"Std:    {np.std(w_dists):.6f}")

In [ ]:
# Plot distribution
plt.figure(figsize=(10, 5))
plt.hist(w_dists, bins=10, alpha=0.7, color='blue', edgecolor='black')
plt.axvline(np.mean(w_dists), color='red', linestyle='--', 
            label=f'Mean: {np.mean(w_dists):.6f}')
plt.xlabel('Wasserstein Distance')
plt.ylabel('Frequency')
plt.title('Distribution of Topological Similarity (English vs French)\nLower = More Similar')
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

## 9. Compare High vs Low Similarity Examples

In [ ]:
# Find most and least topologically similar pairs
sorted_by_similarity = sorted(wasserstein_distances, key=lambda x: x['wasserstein_distance'])

print("Most topologically similar (lowest Wasserstein distance):")
print("="*70)
most_similar = sorted_by_similarity[0]
print(f"Pair {most_similar['idx']}: W-dist = {most_similar['wasserstein_distance']:.6f}")
print(f"  EN: {most_similar['en_text']}")
print(f"  FR: {most_similar['fr_text']}")
print()

print("Least topologically similar (highest Wasserstein distance):")
print("="*70)
least_similar = sorted_by_similarity[-1]
print(f"Pair {least_similar['idx']}: W-dist = {least_similar['wasserstein_distance']:.6f}")
print(f"  EN: {least_similar['en_text']}")
print(f"  FR: {least_similar['fr_text']}")

In [ ]:
# Plot persistence diagrams for most similar pair
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# English
plot_diagram(most_similar['en_diagram'])
plt.suptitle(f"Most Topologically Similar Pair (W-dist: {most_similar['wasserstein_distance']:.6f})")
ax1 = plt.gca()
ax1.set_title('English')

# French  
plt.figure(figsize=(7, 5))
plot_diagram(most_similar['fr_diagram'])
ax2 = plt.gca()
ax2.set_title('French')

plt.tight_layout()
plt.show()

## Summary

✅ **Successfully computed topological features!**

**What we learned:**
- Built distance matrices from attention weights
- Computed Vietoris-Rips persistent homology (β₀, β₁)
- Visualized persistence diagrams
- Measured topological similarity using Wasserstein distance
- Analyzed first 10 sentence pairs

**Next steps:**
1. Scale to all 2000 sentence pairs
2. Compute BLEU scores for translation quality
3. Correlate Wasserstein distance with BLEU scores
4. Statistical analysis and visualization

**Key findings so far:**
- Wasserstein distances range from ~{min} to ~{max}
- Both languages show β₀ and β₁ features
- Topological structure varies across sentence pairs